In [1]:
import findspark
findspark.init()
from pyspark.sql.session import SparkSession

spark = SparkSession.builder\
                    .appName("Titanic Analysis")\
                    .getOrCreate()

print(f"This cluster relies on Spark '{spark.version}'")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


This cluster relies on Spark '3.2.1'


http://blog.madhukaraphatak.com/analysing-kaggle-titanic-data/

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 1500)

In [3]:

from  pyspark.sql.functions import input_file_name

# DataFrame creation    
data = spark.read.option('delimiter',';').csv("hdfs://localhost:9000//datalake/raw/./*/", inferSchema = True, header = True)
                    
                  
# DataFrame updated with an additional column containing the filename contributing to the data in every row.
data = data.withColumn("filename_path", input_file_name())

data.printSchema()

root
 |-- pclass: integer (nullable = true)
 |-- survived: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- ticket: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- cabin: string (nullable = true)
 |-- embarked: string (nullable = true)
 |-- boat: string (nullable = true)
 |-- filename_path: string (nullable = false)



In [4]:
data1= data.select("name", "sex", "age","cabin","fare","ticket","pclass","fare","embarked", 'survived')
data1.printSchema()

root
 |-- name: string (nullable = true)
 |-- sex: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- cabin: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- ticket: string (nullable = true)
 |-- pclass: integer (nullable = true)
 |-- fare: double (nullable = true)
 |-- embarked: string (nullable = true)
 |-- survived: integer (nullable = true)



### Data Types

In [17]:
from  pyspark.sql.functions import explode, min, max, avg, count, col, sum



In [18]:
data1.printSchema()

root
 |-- name: string (nullable = true)
 |-- sex: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- cabin: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- ticket: string (nullable = true)
 |-- pclass: integer (nullable = true)
 |-- fare: double (nullable = true)
 |-- embarked: string (nullable = true)
 |-- survived: integer (nullable = true)



In [19]:
data1.show()

+--------------------+---+----+-------+------+------+------+------+--------+--------+
|                name|sex| age|  cabin|  fare|ticket|pclass|  fare|embarked|survived|
+--------------------+---+----+-------+------+------+------+------+--------+--------+
| Allen, Miss. Eli...|  0|29.0|     B5|211.34| 24160|     1|211.34|       S|       1|
|Allison, Master. ...|  1|0.92|C22 C26|151.55|113781|     1|151.55|       S|       1|
|Allison, Miss. He...|  0| 2.0|C22 C26|151.55|113781|     1|151.55|       S|       0|
|Allison, Mr. Huds...|  1|30.0|C22 C26|151.55|113781|     1|151.55|       S|       0|
|Allison, Mrs. Hud...|  0|25.0|C22 C26|151.55|113781|     1|151.55|       S|       0|
| Anderson, Mr. Harry|  1|48.0|    E12| 26.55| 19952|     1| 26.55|       S|       1|
|Andrews, Miss. Ko...|  0|63.0|     D7| 77.96| 13502|     1| 77.96|       S|       1|
|Andrews, Mr. Thom...|  1|39.0|    A36|   0.0|112050|     1|   0.0|       S|       0|
|Appleton, Mrs. Ed...|  0|53.0|   C101| 51.48| 11769| 

### Import SQL functions

### Questions

#### How to join a two dataframes using JOIN

In [20]:
genderdf= data1.select('name','sex')
agedf   = data1.select('name','age')

genderdf.join(agedf, genderdf.name ==  agedf.name,"leftsemi").show()

+--------------------+---+
|                name|sex|
+--------------------+---+
| Allen, Miss. Eli...|  0|
|Allison, Master. ...|  1|
|Allison, Miss. He...|  0|
|Allison, Mr. Huds...|  1|
|Allison, Mrs. Hud...|  0|
| Anderson, Mr. Harry|  1|
|Andrews, Miss. Ko...|  0|
|Andrews, Mr. Thom...|  1|
|Appleton, Mrs. Ed...|  0|
|Artagaveytia, Mr....|  1|
|Astor, Col. John ...|  1|
|Astor, Mrs. John ...|  0|
|Aubart, Mme. Leon...|  0|
|"Barber, Miss. El...|  0|
|Barkworth, Mr. Al...|  1|
|Baxter, Mr. Quigg...|  1|
|Baxter, Mrs. Jame...|  0|
|Bazzani, Miss. Al...|  0|
|Beattie, Mr. Thomson|  1|
|Beckwith, Mr. Ric...|  1|
+--------------------+---+
only showing top 20 rows



In [56]:
length = data1.count()

#### What is the average fare  by class and sex?

In [21]:
data1.select('pclass', 'sex', 'fare')\
.groupBy('sex','pclass')\
.avg('fare')\
.orderBy('sex','pclass')\
.toPandas()

,sex,pclass,avg(fare)
0,0,1,112.486031
1,0,2,23.267379
2,0,3,14.656908
3,1,1,74.818477
4,1,2,20.934430
5,1,3,12.139683


##### First class tickets 

#####  Women = 0, Men = 1

#### Male and female breakdown

In [57]:
data1.groupBy('sex').count().withColumn('ratio', col('count')/length).toPandas()


,sex,count,ratio
0,1,656,0.629559
1,0,386,0.370441


#### Male and female breakdown from those who survived

In [70]:
data1.select('sex','survived').groupBy("sex", 'survived').count().toPandas()


,sex,survived,count
0,1,0,522
1,1,1,134
2,0,0,96
3,0,1,290


#### How many passengers bought tickets for each class?

In [64]:
data1.groupBy('pclass').count().withColumn('ratio', col('count')/length).orderBy('pclass').toPandas()



,pclass,count,ratio
0,1,282,0.270633
1,2,261,0.250480
2,3,499,0.478887


#### How many women survived from each class?

In [61]:
survival_female = data1.select('survived','sex','pclass').groupBy('pclass','sex', 'survived').count()\
.withColumn('ratio', col('count')/length)\
.where('sex = 0')\
.where('survived =1')\
.orderBy('pclass')
survival_female.toPandas()

,pclass,sex,survived,count,ratio
0,1,0,1,126,0.120921
1,2,0,1,92,0.088292
2,3,0,1,72,0.069098


#### How many men survived from each class?

In [62]:
survival_male = data1.select('survived','sex','pclass').groupBy('pclass','sex', 'survived').count() \
.withColumn('ratio', col('count')/length)\
.where('sex = 1')\
.where('survived =1')\
.orderBy('pclass')
survival_male.toPandas()

,pclass,sex,survived,count,ratio
0,1,1,1,53,0.050864
1,2,1,1,23,0.022073
2,3,1,1,58,0.055662
